In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
pd.set_option('display.max_colwidth',2500)
pd.set_option('display.max_columns',500)

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            if str(col_type) == numerics:
                c_min = df[col].min()
                c_max = df[col].max()

                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            if str(col_type)[:5] == 'float':
                c_min = df[col].min()
                c_max = df[col].max()

                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

#import gc

#df = reduce_mem_usage(df)

#gc.collect()

In [4]:
def preprocess_frame(fr):
    fr = fr.sort_values(['vid_id','session_id','frame','vidTime','timelineRatio'])
    fr = (fr.reset_index()).drop('index',axis=1)
    fr = fr.reset_index()
    fr['index'] = fr['index']-(fr['index']%1000)
    fr['vid_count'] = fr.groupby(['vid_id'])['vid_id'].transform('count')
    fr['session_count'] = fr.groupby(['vid_id','session_id'])['session_id'].transform('count')
    fr['session_mean'] = fr.groupby(['session_id'])['vid_count'].transform('mean')
    fr['vid_mean'] = fr.groupby(['vid_id'])['session_mean'].transform('mean')
    fr['count_ratio'] = fr['vid_mean']/fr['session_mean']
    #fr['count_diff'] = fr['vid_mean']-fr['session_mean']
    #fr = reduce_mem_usage(fr)
    for col in ['frame']:#,'vidTime','timelineRatio']:
        fr[col+'diff'] = fr.groupby(['vid_id','session_id','index'])[col].shift(1)
        fr[col+'diff'] = fr[col] - fr[col+'diff']
        fr[col+'std'] = fr.groupby(['vid_id','session_id'])[col+'diff'].transform('std')
        #fr[col+'min'] = fr.groupby(['vid_id','session_id'])[col+'diff'].transform('min')
        #fr[col+'max'] = fr.groupby(['vid_id','session_id'])[col+'diff'].transform('max')
        fr[col+'stdmean'] = fr.groupby('vid_id')[col+'std'].transform('mean')
    fr = reduce_mem_usage(fr)
    fr = (fr.groupby('vid_id').mean()).reset_index()
    #fr = fr.drop_duplicates('vid_id')
    return fr

In [5]:
fr = pd.read_csv('large_files/test_frames1/test_frames1.csv')
fr = reduce_mem_usage(fr)
fr = preprocess_frame(fr)
fr.to_csv('fr3.csv',index=False)
fr